In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.functional as F
from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup, get_constant_schedule_with_warmup
import re
from tqdm import tqdm_notebook
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, confusion_matrix, accuracy_score
import ast
import copy

from tqdm.auto import tqdm, trange
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
model_checkpoint = "seara/rubert-tiny2-russian-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
model.classifier = torch.nn.Linear(312, 5)

In [39]:
aliance = pd.read_csv('../data/new_names_and_synonyms_i_already_letter_maybe.csv', index_col=0)
aliance

,issuerid,EMITENT_FULL_NAME,combined
0,1,"Акционерный коммерческий банк ""Держава"" публич...","['Держава', 'DERZP', 'Державы', 'DERZHAVA', 'D..."
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...","['МОСКОВСКИЙ КРЕДИТНЫЙ БАНК', 'CBOM RX', 'Моск..."
2,3,"""Российский акционерный коммерческий дорожный ...",['Российский акционерный коммерческий дорожный...
3,4,"Акционерная компания ""АЛРОСА"" (публичное акцио...","['АЛРОСА', 'ALRS RX', 'алросы', 'alrosa']"
4,5,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...","['АВАНГАРД', 'AVAN', 'Авангарда', 'AVANGARD']"
...,...,...,...
250,270,Henderson,"['Henderson', 'HNFG', 'Хендерсон', 'Хендерсона']"
251,271,Совкомбанк,"['Совкомбанк', 'SVCB', 'Sovcombank', 'Совкомба..."
252,272,ЕвроТранс,"['ЕвроТранс', 'EUTR', 'АЗС Трасса', 'АЗС ""Трас..."
253,273,Делимобиль,"['Делимобиль', 'DELI', 'Делимобил', 'Каршеринг..."


In [40]:
sentiment_data = pd.read_csv('../data/concat.csv')
sentiment_data = sentiment_data.dropna()
sentiment_data

,SentimentScore,MessageText,combined,EMITENT_FULL_NAME
0,2,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,SELG ; селигдар,"Публичное акционерное общество ""Селигдар"""
1,4,Ozon продолжает развивать специализированные ф...,OZON RX,Озон Холдингс ПиЭлСи (эмитент депозитарных рас...
2,4,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,nmtp; нмтп; новороссийский морской порт; новор...,"Публичное акционерное общество ""Новороссийский..."
3,5,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,GLTR LI; Globaltrans; Глобалтранс,Globaltrans Investment PLC (Глобалтранс Инвест...
4,2,​​Windfall Tax — налог на сверхприбыль. Какие ...,MGNT RX; Магнит ; Magnit ; Mgnt ; ПАО Магнит; ...,"Публичное акционерное общество ""Магнит"""
...,...,...,...,...
9284,4,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,FLOT RX; совкомфлот,"Публичное акционерное общество ""Современный ко..."
9285,4,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,FLOT RX; совкомфлот,"Публичное акционерное общество ""Современный ко..."
9286,3,​​Ключевой принцип создания портфеля 🔹Диверси...,TCS LI; TCSG; TCS Group; Tinkoff; Тинькофф,TCS Group Holding PLC (ТиСиЭс Груп Холдинг ПиЭ...
9287,3,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",PLZL RX; Полюс,"Публичное акционерное общество ""Полюс"""


In [41]:
def clean_txt(text):
    text = text.lower()
    # Заменяем символы форматирования на пустую строку
    cleaned_text = re.sub(r'[*_]', '', text)
    cleaned_text = re.sub(r'\n', '', cleaned_text)
    return cleaned_text

sentiment_data.MessageText = sentiment_data.MessageText.apply(lambda x: clean_txt(x))
sentiment_data

,SentimentScore,MessageText,combined,EMITENT_FULL_NAME
0,2,⚠️🇷🇺#selg #дивиденд сд селигдар: дивиденды 20...,SELG ; селигдар,"Публичное акционерное общество ""Селигдар"""
1,4,ozon продолжает развивать специализированные ф...,OZON RX,Озон Холдингс ПиЭлСи (эмитент депозитарных рас...
2,4,​фокусы продолжаются🔥акции и инвестиции 📈втб ...,nmtp; нмтп; новороссийский морской порт; новор...,"Публичное акционерное общество ""Новороссийский..."
3,5,​фокусы продолжаются🔥акции и инвестиции 📈втб ...,GLTR LI; Globaltrans; Глобалтранс,Globaltrans Investment PLC (Глобалтранс Инвест...
4,2,​​windfall tax — налог на сверхприбыль. какие ...,MGNT RX; Магнит ; Magnit ; Mgnt ; ПАО Магнит; ...,"Публичное акционерное общество ""Магнит"""
...,...,...,...,...
9284,4,#flot #дивиденды 💰 7% — возможная дивдоходност...,FLOT RX; совкомфлот,"Публичное акционерное общество ""Современный ко..."
9285,4,🇷🇺#flot #отчетность чистая прибыль совкомфлот...,FLOT RX; совкомфлот,"Публичное акционерное общество ""Современный ко..."
9286,3,​​ключевой принцип создания портфеля 🔹диверси...,TCS LI; TCSG; TCS Group; Tinkoff; Тинькофф,TCS Group Holding PLC (ТиСиЭс Груп Холдинг ПиЭ...
9287,3,"""💥🇷🇺#plzl #листинг #торги """"полюс"""" ведет диа...",PLZL RX; Полюс,"Публичное акционерное общество ""Полюс"""


In [42]:
sentiment_data = sentiment_data.merge(aliance, on='EMITENT_FULL_NAME')
sentiment_data

,SentimentScore,MessageText,combined_x,EMITENT_FULL_NAME,issuerid,combined_y
0,2,⚠️🇷🇺#selg #дивиденд сд селигдар: дивиденды 20...,SELG ; селигдар,"Публичное акционерное общество ""Селигдар""",153,"['Селигдар', 'SELG', 'селигдара']"
1,3,❗️🇷🇺🇺🇸 #рынки #ожидания #россия #сша нефтегаз...,SELG ; селигдар,"Публичное акционерное общество ""Селигдар""",153,"['Селигдар', 'SELG', 'селигдара']"
2,3,"⚖️ полюс, polymetal, селигдар. оцениваем их ак...",SELG ; селигдар,"Публичное акционерное общество ""Селигдар""",153,"['Селигдар', 'SELG', 'селигдара']"
3,3,"""скорректированная ebitda холдинга """"селигдар""...",SELG ; селигдар,"Публичное акционерное общество ""Селигдар""",153,"['Селигдар', 'SELG', 'селигдара']"
4,3,события рф: 🗓 понедельник 🗓 вторник ◾️ $hy...,SELG ; селигдар,"Публичное акционерное общество ""Селигдар""",153,"['Селигдар', 'SELG', 'селигдара']"
...,...,...,...,...,...,...
9280,3,💥🇷🇺#igst = +15%,IGST ; Ижсталь; Ижевский металл...,"Публичное акционерное общество ""Ижсталь""",67,"['Ижсталь', 'IGST', 'Ижстали', 'Ижевский метал..."
9281,3,💥🇷🇺#igst = +25%,IGST ; Ижсталь; Ижевский металл...,"Публичное акционерное общество ""Ижсталь""",67,"['Ижсталь', 'IGST', 'Ижстали', 'Ижевский метал..."
9282,4,🇷🇺#msrs #отчетность чистая прибыль россети мо...,MSRS ; Россети Московский регион,"Публичное акционерное общество ""Россети Москов...",137,"['Россети Московский регион', 'MSRS', 'Россети..."
9283,3,💥🇷🇺#msrs = макс за 7 мес,MSRS ; Россети Московский регион,"Публичное акционерное общество ""Россети Москов...",137,"['Россети Московский регион', 'MSRS', 'Россети..."


In [43]:
sentiment_data.first_alias = sentiment_data.combined_y.apply(lambda x: ast.literal_eval(x)[0])
sentiment_data.combined_y = sentiment_data.combined_y.apply(lambda x: ast.literal_eval(x))
sentiment_data['MessageText_SEP'] = sentiment_data.first_alias.apply(lambda x: x.lower()) + '[SEP]' + sentiment_data.MessageText
sentiment_data['MessageText_SEP']

/tmp/ipykernel_97954/864856943.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sentiment_data.first_alias = sentiment_data.combined_y.apply(lambda x: ast.literal_eval(x)[0])


0       селигдар[SEP]⚠️🇷🇺#selg #дивиденд  сд селигдар:...
1       селигдар[SEP]❗️🇷🇺🇺🇸 #рынки #ожидания #россия #...
2       селигдар[SEP]⚖️ полюс, polymetal, селигдар. оц...
3       селигдар[SEP]"скорректированная ebitda холдинг...
4       селигдар[SEP]события рф:  🗓 понедельник   🗓 вт...
                              ...                        
9280                          ижсталь[SEP]💥🇷🇺#igst = +15%
9281                          ижсталь[SEP]💥🇷🇺#igst = +25%
9282    россети московский регион[SEP]🇷🇺#msrs #отчетно...
9283    россети московский регион[SEP]💥🇷🇺#msrs = макс ...
9284    башинформсвязь[SEP]🇷🇺#bisv #отчетность  башинф...
Name: MessageText_SEP, Length: 9285, dtype: object

In [44]:
count = 0
new_message = []
def replace_on_token(text, names_company):
    global count
    for name in names_company:
        old_text = copy.copy(text)
        name = name.lower()
        text = text.replace(name, "[COMP]")
        text = text.replace(name.split()[0], "[COMP]")
    return text
for i in tqdm(range(sentiment_data.shape[0])):
    sample = sentiment_data.iloc[i]
    post, name_company = sample.MessageText, sample.combined_y
    post = replace_on_token(post, name_company)
    new_message.append(post)
sentiment_data['MessageText_COMP'] = new_message
print(f'{count}/{sentiment_data.shape[0]}')   

100%|████████████████████████████████████| 9285/9285 [00:00<00:00, 21097.24it/s]

0/9285


In [45]:
sentiment_data = sentiment_data.drop(['combined_y','MessageText', 'combined_x', 'EMITENT_FULL_NAME', 'issuerid'], axis=1)
sentiment_data

,SentimentScore,MessageText_SEP,MessageText_COMP
0,2,селигдар[SEP]⚠️🇷🇺#selg #дивиденд сд селигдар:...,⚠️🇷🇺#[COMP] #дивиденд сд [COMP]: дивиденды 20...
1,3,селигдар[SEP]❗️🇷🇺🇺🇸 #рынки #ожидания #россия #...,❗️🇷🇺🇺🇸 #рынки #ожидания #россия #сша нефтегаз...
2,3,"селигдар[SEP]⚖️ полюс, polymetal, селигдар. оц...","⚖️ полюс, polymetal, [COMP]. оцениваем их акци..."
3,3,"селигдар[SEP]""скорректированная ebitda холдинг...","""скорректированная ebitda холдинга """"[COMP]"""" ..."
4,3,селигдар[SEP]события рф: 🗓 понедельник 🗓 вт...,события рф: 🗓 понедельник 🗓 вторник ◾️ $hy...
...,...,...,...
9280,3,ижсталь[SEP]💥🇷🇺#igst = +15%,💥🇷🇺#[COMP] = +15%
9281,3,ижсталь[SEP]💥🇷🇺#igst = +25%,💥🇷🇺#[COMP] = +25%
9282,4,россети московский регион[SEP]🇷🇺#msrs #отчетно...,🇷🇺#[COMP] #отчетность чистая прибыль [COMP] п...
9283,3,россети московский регион[SEP]💥🇷🇺#msrs = макс ...,💥🇷🇺#[COMP] = макс за 7 мес


MessageText_SEP - текста для модели, в которой мы используем [SEP] токен, им отделяем название компании,
которое используем для сентимент анализа
MessageText_COMP - текста для модели, в которой мы заменяем названия нужной нам компании токеном [COMP]

In [46]:
sentiment_data = sentiment_data[sentiment_data['SentimentScore'] != 0]
sentiment_data['SentimentScore'] = sentiment_data['SentimentScore'].apply(lambda x: x-1)
sentiment_data

/tmp/ipykernel_97954/1074956094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_data['SentimentScore'] = sentiment_data['SentimentScore'].apply(lambda x: x-1)


,SentimentScore,MessageText_SEP,MessageText_COMP
0,1,селигдар[SEP]⚠️🇷🇺#selg #дивиденд сд селигдар:...,⚠️🇷🇺#[COMP] #дивиденд сд [COMP]: дивиденды 20...
1,2,селигдар[SEP]❗️🇷🇺🇺🇸 #рынки #ожидания #россия #...,❗️🇷🇺🇺🇸 #рынки #ожидания #россия #сша нефтегаз...
2,2,"селигдар[SEP]⚖️ полюс, polymetal, селигдар. оц...","⚖️ полюс, polymetal, [COMP]. оцениваем их акци..."
3,2,"селигдар[SEP]""скорректированная ebitda холдинг...","""скорректированная ebitda холдинга """"[COMP]"""" ..."
4,2,селигдар[SEP]события рф: 🗓 понедельник 🗓 вт...,события рф: 🗓 понедельник 🗓 вторник ◾️ $hy...
...,...,...,...
9280,2,ижсталь[SEP]💥🇷🇺#igst = +15%,💥🇷🇺#[COMP] = +15%
9281,2,ижсталь[SEP]💥🇷🇺#igst = +25%,💥🇷🇺#[COMP] = +25%
9282,3,россети московский регион[SEP]🇷🇺#msrs #отчетно...,🇷🇺#[COMP] #отчетность чистая прибыль [COMP] п...
9283,2,россети московский регион[SEP]💥🇷🇺#msrs = макс ...,💥🇷🇺#[COMP] = макс за 7 мес


In [47]:
train, test = train_test_split(sentiment_data, test_size=0.15, random_state=42)

In [48]:
torch_data_COMP = DatasetDict({
    'train': Dataset.from_pandas(train.drop(['MessageText_SEP'], axis=1).reset_index(drop=True)),
    'test': Dataset.from_pandas(test.drop(['MessageText_SEP'], axis=1).reset_index(drop=True))
})
torch_data_COMP

DatasetDict({
    train: Dataset({
        features: ['SentimentScore', 'MessageText_COMP'],
        num_rows: 7753
    })
    test: Dataset({
        features: ['SentimentScore', 'MessageText_COMP'],
        num_rows: 1369
    })
})

In [49]:
torch_data_SEP = DatasetDict({
    'train': Dataset.from_pandas(train.drop(['MessageText_COMP'], axis=1).reset_index(drop=True)),
    'test': Dataset.from_pandas(test.drop(['MessageText_COMP'], axis=1).reset_index(drop=True))
})
torch_data_SEP

DatasetDict({
    train: Dataset({
        features: ['SentimentScore', 'MessageText_SEP'],
        num_rows: 7753
    })
    test: Dataset({
        features: ['SentimentScore', 'MessageText_SEP'],
        num_rows: 1369
    })
})

In [50]:
all_labels = [0, 1, 2, 3, 4]

In [51]:
data_collator = DataCollatorWithPadding(tokenizer)

In [52]:
model_comp = model
model_sep = model

In [53]:
tokenizer.add_tokens(['[COMP]'])
model_comp.resize_token_embeddings(len(tokenizer))

Embedding(83829, 312)

In [54]:
data_tokenized_SEP = torch_data_SEP.map(
    lambda x: tokenizer(x["MessageText_SEP"], truncation=True, padding=True, max_length=512), batched=True, remove_columns=['MessageText_SEP']
)
data_tokenized_SEP = data_tokenized_SEP.map(lambda x: {'sentiment': [all_labels.index(xl) for xl in x['SentimentScore']]}, batched=True)


Map: 100%|███████████████████████| 1369/1369 [00:00<00:00, 220490.06 examples/s]


In [55]:
data_tokenized_COMP = torch_data_COMP.map(
    lambda x: tokenizer(x["MessageText_COMP"], truncation=True, padding=True, max_length=512), batched=True, remove_columns=['MessageText_COMP']
)
data_tokenized_COMP = data_tokenized_COMP.map(lambda x: {'sentiment': [all_labels.index(xl) for xl in x['SentimentScore']]}, batched=True)


Map: 100%|███████████████████████| 1369/1369 [00:00<00:00, 221425.35 examples/s]


Тут решаем проблему дисбаланса классов и добавляем WeightedRandomSampler

In [56]:
# # Получение уникальных меток классов и их количества в тренировочном наборе данных
unique_labels, counts = np.unique(data_tokenized_COMP['train']['SentimentScore'], return_counts=True)

# Вычисление весов классов
class_weights = compute_class_weight('balanced', classes=unique_labels, y=data_tokenized_COMP['train']['SentimentScore'])

# Создание словаря весов для каждого класса
class_weights_dict = dict(zip(unique_labels, class_weights))

print("Веса классов:", class_weights_dict)

from torch.utils.data import WeightedRandomSampler

weights = [class_weights_dict[label] for label in data_tokenized_COMP['train']['SentimentScore']]

# Создание WeightedRandomSampler
sampler = WeightedRandomSampler(weights, len(train), replacement=True)


batch_size = 64

train_dataloader_COMP = DataLoader(
    data_tokenized_COMP['train'],
    batch_size=batch_size, drop_last=False, num_workers=0, collate_fn=data_collator,
    sampler=sampler
)
train_dataloader_SEP = DataLoader(
    data_tokenized_SEP['train'],
    batch_size=batch_size, drop_last=False, num_workers=0, collate_fn=data_collator,
    sampler=sampler
)

test_dataloader_SEP = DataLoader(
    data_tokenized_SEP['test'],
    batch_size=batch_size, drop_last=False, num_workers=0, collate_fn=data_collator,
)
test_dataloader_COMP = DataLoader(
    data_tokenized_COMP['test'],
    batch_size=batch_size, drop_last=False, num_workers=0, collate_fn=data_collator,
)

Веса классов: {0: 29.81923076923077, 1: 1.9214374225526643, 2: 0.5210349462365591, 3: 0.4743346589171, 4: 2.3892141756548537}


In [23]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
def calculate_accuracy(dataloader, model):
    """
    Вычисляет точность модели на основе даталоадера.
    
    Параметры:
    - dataloader: объект DataLoader, содержащий данные для тестирования
    - model: обученная модель PyTorch
    
    Возвращает:
    - accuracy: доля правильных предсказаний
    """
    # Переводим модель в режим оценки
    model.eval()
    
    # Инициализируем счетчик правильных предсказаний
    correct = 0
    total = 0
    
    # Итерируемся по всем батчам данных
    with torch.no_grad():
        for data in dataloader:
            # Переводим данные и метки на устройство, на котором работает модель
            data, labels = data.input_ids.to(model.device), data.SentimentScore.to(model.device)
            
            # Получаем предсказания модели
            outputs = model(data).logits
            
            # Находим индексы максимальных значений в предсказаниях
#             _, predicted = torch.max(outputs, 1)
            predicted = outputs.argmax(1)
            
            # Считаем количество правильных предсказаний
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    # Вычисляем среднее значение (точность)
    accuracy = correct / total
    
    return accuracy

In [22]:
def evaluate_model(model, dev_dataloader, verbose=False, labels=None):
    facts, preds = predict_with_model(model, dev_dataloader)
    pfrs, aucs = get_classification_report(facts, preds, labels)
    if verbose:
        display(pfrs)
        print('aucs:', aucs, np.mean(aucs))
    return np.mean(aucs)

def predict_with_model(model, dataloader):
    preds = []
    facts = []

    for batch in tqdm(dataloader):
        facts.append(batch.sentiment.cpu().numpy())
        batch = batch.to(model.device)
        with torch.no_grad():
            pr = model(input_ids=batch.input_ids, attention_mask=batch.attention_mask, token_type_ids=batch.token_type_ids)
        preds.append(torch.softmax(pr.logits, -1).cpu().numpy())
    facts = np.concatenate(facts)
    preds = np.concatenate(preds)
    return facts, preds

def get_classification_report(facts, preds, labels=None):
    pfrs = pd.DataFrame(dict(zip(['Precision', 'Recall', 'F1', 'Sum'], precision_recall_fscore_support(facts, preds.argmax(1)))))
    aucs = [roc_auc_score(facts==i, preds[:, i]) for i in set(facts)]
    pfrs['AUC'] = aucs
    pfrs = pd.concat([pfrs, pfrs.mean().to_frame().T], ignore_index=True)
    if labels is not None:
        pfrs.index = list(labels) + ['mean']
    return pfrs, aucs

def evaluate_model_matrix(model, dev_dataloader, verbose=False, labels=[0, 1, 2, 3, 4, 5]):
    facts, preds = predict_with_model(model, dev_dataloader)
#     return facts, preds
    pfrs, aucs = get_classification_report(facts, preds, labels)
    
    # Вычисление confusion matrix для каждого класса
    conf_matrices = {}
    for i, label in enumerate(labels):
        conf_matrices[label] = confusion_matrix(facts, preds.argmax(1), labels=[i])
        if verbose:
            plt.figure(figsize=(6, 4))
            sns.heatmap(conf_matrices[label], annot=True, fmt='d', cmap='Blues')
            plt.xlabel('Predicted labels')
            plt.ylabel('True labels')
            plt.title(f'Confusion Matrix for class {label}')
            plt.show()
    
    if verbose:
        display(pfrs)
        print('aucs:', aucs, np.mean(aucs))
    
    return np.mean(aucs), conf_matrices

In [ ]:
evaluate_model(model, test_dataloader_COMP, verbose=True)

In [24]:
gradient_accumulation_steps = 8
window = 500
cleanup_step = 100
report_step = 10000
ewm_loss = 0

In [25]:
import wandb

wandb.login()

wandb: Currently logged in as: danil-dushenev. Use `wandb login --relogin` to force relogin


True

In [26]:
def train_loop(model, optimizer, scheduler, criterion, train_dataloader, test_dataloader,
         warmup_steps, total_steps, initial_lr, epochs=30, model_name="main-solution"):
    wandb.init(project="gagarin-hack-sentiment-bert", name=model_name)
    
    best_val_auc = float(0)
    best_val_acc = float(0)

    all_preds = []
    all_labels = []
    
    save_emb = []
    model = model.to(device)
    best_model = model

    for epoch in trange(epochs):
        tq = tqdm(train_dataloader)
        mean_loss = 0
        for i, batch in enumerate(tq):
            try:
                batch = batch.to(device)
                output = model(batch.input_ids, attention_mask=batch.attention_mask)
                logits = output.logits  # Получаем оценки вероятностей классов
                labels = batch.sentiment  # Получаем целевые метки
                loss = criterion(logits, labels)  # Вычисляем CrossEntropyLoss
                loss.backward()
            except RuntimeError as e:
                print('error on step', i, e)
                loss = None
                continue
    
            if i and i % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                
            mean_loss += loss.item()
            w = 1 / min(i+1, window)
            ewm_loss = ewm_loss * (1-w) + loss.item() * w
            tq.set_description(f'loss: {loss.item():4.4f}')
    
        # Шаг планировщика после каждой эпохи
        scheduler.step()
        mean_loss /= len(train_dataloader)
        model.eval()
        val_accuracy = calculate_accuracy(test_dataloader, model)
        eval_loss = evaluate_model(model, test_dataloader, verbose=True)
        wandb.log({"train_loss": ewm_loss, "val_aсc": val_accuracy})
        model.train()
        print(f'epoch {epoch + 1}, step {i}: train loss: {mean_loss:4.4f}  val acc: {val_accuracy}')
        
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            torch.save(model, f'{model_name}_model_{val_accuracy}.pth')


In [ ]:
warmup_steps = 1 * 124
total_steps = 30*124
initial_lr = 1e-3

optimizer = torch.optim.Adam(params=model.parameters(), lr=initial_lr)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)  
criterion = nn.CrossEntropyLoss()


train_loop(model_sep, optimizer, scheduler, criterion, train_dataloader_SEP, test_dataloader_SEP,
         warmup_steps, total_steps, initial_lr, epochs=30, model_name="bert_tiny_sep")

  0%|                                                   | 0/122 [00:00<?, ?it/s]

In [ ]:
warmup_steps = 1 * 124
total_steps = 30*124
initial_lr = 1e-3

optimizer = torch.optim.Adam(params=model.parameters(), lr=initial_lr)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)  
criterion = nn.CrossEntropyLoss()


train_loop(model_comp, optimizer, scheduler, criterion, train_dataloader_COMP, test_dataloader_COMP,
         warmup_steps, total_steps, initial_lr, epochs=30, model_name="bert_tiny_COMP")

In [57]:
model_comp = AutoModelForSequenceClassification.from_pretrained('../weights/DENCHIK3000/', local_files_only=True)
model_sep = AutoModelForSequenceClassification.from_pretrained('../weights/sentiment_quant_model/', local_files_only=True)

Detected the presence of a `quantization_config` attribute in the model's configuration but you don't have the correct `bitsandbytes` version to support 4 and 8 bit serialization. Please install the latest version of `bitsandbytes` with  `pip install --upgrade bitsandbytes`.
Some weights of the model checkpoint at ../weights/DENCHIK3000/ were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.0.attention.output.dense.SCB', 'bert.encoder.layer.0.attention.output.dense.weight_format', 'bert.encoder.layer.0.attention.self.key.SCB', 'bert.encoder.layer.0.attention.self.key.weight_format', 'bert.encoder.layer.0.attention.self.query.SCB', 'bert.encoder.layer.0.attention.self.query.weight_format', 'bert.encoder.layer.0.attention.self.value.SCB', 'bert.encoder.layer.0.attention.self.value.weight_format', 'bert.encoder.layer.0.intermediate.dense.SCB', 'bert.encoder.layer.0.intermediate.dense.weight_format', 'bert.encoder.layer.0.output.dense.SCB', 'bert.encoder.layer

In [58]:
def save_model_out(dataloader, model):
    all_outputs = []
    all_labels = []

    # Перевод модели в режим оценки
    model.eval()

    # Итерация по всем батчам данных
    for inputs in dataloader:
        # Перевод данных на устройство, на котором работает модель
        inputs, labels = inputs.input_ids.to(model.device), inputs.sentiment.to(model.device)

        # Выполнение прямого прохода
        with torch.no_grad():
            outputs = model(inputs)

        # Добавление выходов в список
        all_outputs.append(outputs.logits.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
        

    # Преобразование списка выходов в массив NumPy для удобства работы
    all_outputs = np.concatenate(all_outputs, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    return all_outputs, all_labels

COMP_logits, labels = save_model_out(test_dataloader_COMP, model_comp)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
SEP_logits, _ = save_model_out(test_dataloader_SEP, model_sep)

In [ ]:
blend = (SEP_logits + COMP_logits) / 2
sum(blend.argmax(-1) == labels) / len(labels)
# ~ 0.68 accuracy